In [1]:
import _pickle as pickle
import pandas as pd
import numpy as np
import re

In [2]:
train_eval = pickle.load(open('../data/single_turn_train_eval.pkl', 'rb'))

In [16]:
train_eval.keys()

dict_keys(['NDCG1', 'NDCG3', 'NDCG5', 'NDCG10', 'NDCG20', 'P1', 'P3', 'P5', 'P10', 'P20', 'MRR100'])

In [18]:
train_eval['NDCG3']

{'F0513': {'Q00045': {'no_answer': 0.3086598407528446,
   'with_answer': 0.3086598407528446},
  'Q03825': {'no_answer': 0.2653606369886217,
   'with_answer': 0.3673196815056891},
  'Q00451': {'no_answer': 0.1759795222585337,
   'with_answer': 0.3086598407528446},
  'Q00493': {'no_answer': 0.0, 'with_answer': 0.3086598407528446},
  'Q00621': {'no_answer': 0.3086598407528446,
   'with_answer': 0.3673196815056891},
  'Q02132': {'no_answer': 0.2932992037642228,
   'with_answer': 0.3673196815056891},
  'Q02303': {'no_answer': 0.23463936301137828,
   'with_answer': 0.3673196815056891},
  'Q02369': {'no_answer': 0.4413401592471554,
   'with_answer': 0.4413401592471554},
  'Q02392': {'no_answer': 0.23463936301137828,
   'with_answer': 0.2932992037642228},
  'Q03736': {'no_answer': 0.2932992037642228,
   'with_answer': 0.3673196815056891},
  'Q00001': {'with_answer': 0.3673196815056891,
   'no_answer': 0.3673196815056891},
  'MAX': {'with_answer': 0.4413401592471554, 'no_answer': 0.441340159247

In [3]:
trainset = pd.read_csv('../data/train.tsv', sep='\t')
devset = pd.read_csv('../data/dev.tsv', sep='\t')

In [39]:
def get_answer_scores(trainset):
#     score_keys = list(train_eval.keys())
    score_keys = ['P5']
    answer_scores = []
    for fid, ir, qid, question, answer in trainset[['facet_id', 'initial_request', 'question_id', 'question', 'answer']].values:
        if pd.isna(answer):
            continue
        scores = {}
        for k in score_keys:
            if fid in train_eval[k] and qid in train_eval[k][fid]:
                scores[k] = train_eval[k][fid][qid]['with_answer']
            else:
                scores[k] = 0.
        answer_scores.append((ir, question, answer, scores, np.mean([_v for _v in scores.values()])))
    return answer_scores

In [40]:
train_answer_scores = get_answer_scores(trainset)
print(len(train_answer_scores), len([_a for _a in train_answer_scores if _a[-1] == 0]))

buckets = [0 for _ in range(11)]
for a in train_answer_scores:
    if a[-1] == 0:
        i = 0
    else:
        i = int(a[-1] * 10)
        i = i if i < 10 else 9
        i += 1
    buckets[i] += 1
print(buckets)

8566 4895
[4895, 0, 0, 1373, 0, 816, 0, 629, 0, 430, 423]


In [42]:
dev_answer_scores = get_answer_scores(devset)
print(len(dev_answer_scores), len([_a for _a in dev_answer_scores if _a[-1] == 0]))

buckets = [0 for _ in range(11)]
for a in dev_answer_scores:
    if a[-1] == 0:
        i = 0
    else:
        i = int(a[-1] * 10)
        i = i if i < 10 else 9
        i += 1
    buckets[i] += 1
print(buckets)

2161 1258
[1258, 0, 0, 346, 0, 185, 0, 149, 0, 127, 96]


In [43]:
def build_datas(answer_scores):
    datas = []
    for a in answer_scores:
        if a[-1] == 0:
            datas.append((a[0], a[1], a[2], 0))
        else:
            datas.append((a[0], a[1], a[2], 1))
    return datas

In [44]:
train_answer_datas = build_datas(train_answer_scores)
print(len([_d for _d in train_answer_datas if _d[-1] == 0]))
print(len([_d for _d in train_answer_datas if _d[-1] == 1]))

4895
3671
0


In [45]:
dev_answer_datas = build_datas(dev_answer_scores)
print(len([_d for _d in dev_answer_datas if _d[-1] == 0]))
print(len([_d for _d in dev_answer_datas if _d[-1] == 1]))
print(len([_d for _d in dev_answer_datas if _d[-1] == 2]))

1258
903
0


In [46]:
pickle.dump(train_answer_datas, open('../data/answer_datas/train.pkl', 'wb'))
pickle.dump(dev_answer_datas, open('../data/answer_datas/dev.pkl', 'wb'))

In [47]:
train_answer_scores

[('Tell me about Obama family tree.',
  'are you interested in seeing barack obamas family',
  'yes am interested in obamas family',
  {'P5': 0.0},
  0.0),
 ('Tell me about Obama family tree.',
  'would you like to know barack obamas geneology',
  'yes i want to know who made up his family',
  {'P5': 0.0},
  0.0),
 ('Tell me about Obama family tree.',
  'would you like to know about obamas ancestors',
  'yes this is what am looking for',
  {'P5': 0.0},
  0.0),
 ('Tell me about Obama family tree.',
  'would you like to know who is currently alive from president obamas family tree',
  'no find the barack obamas family tree time magazine photo essay',
  {'P5': 0.0},
  0.0),
 ('Tell me about Obama family tree.',
  'are you looking for biological information on his family',
  'no i need the specific time magazine article titled barack obamas family tree',
  {'P5': 0.0},
  0.0),
 ('Tell me about Obama family tree.',
  'are you referring to the time magazine essay',
  'yes please locate the p

In [48]:
dev_answer_datas

[('Find me information about the Ritz Carlton Lake Las Vegas.',
  'are you looking for a specific web site',
  'yes for the ritz carlton resort at lake las vegas',
  1),
 ('Find me information about the Ritz Carlton Lake Las Vegas.',
  'would you like the history of ritz carlton lake las vegas',
  'where can i find the history of the ritz carton lake in las vegas',
  1),
 ('Find me information about the Ritz Carlton Lake Las Vegas.',
  'would you like the location of the ritz carlton lake las vegas',
  'yes along with other information',
  1),
 ('Find me information about the Ritz Carlton Lake Las Vegas.',
  'would you like to know the capacity of ritz carlton lake las vegas',
  'yes and other information',
  1),
 ('Find me information about the Ritz Carlton Lake Las Vegas.',
  'would you like to know where ritz carlton lake las vegas is on a map',
  'no i want to know more about the ritz carlton lake las vegas resort',
  1),
 ('Find me information about the Ritz Carlton Lake Las Vegas